In [10]:
# !git clone https://github.com/tiaoteek/fasterRCNN
# %cd fasterRCNN
# %pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output  # to display images
clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cpu (CPU)


In [9]:
# !rm -rf /content/fasterRCNN/fasterRCNN

In [2]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

100%|██████████| 780M/780M [02:29<00:00, 5.48MB/s]
'unzip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [14]:
import torchvision  # 0.3.0  version  这里指的是所使用包的版本
from torchvision import transforms as T
import cv2  # 4.1.1  version
import matplotlib.pyplot as plt  # 3.0.0  version
from PIL import Image  # 5.3.0  version
import random
import os
import torch
import numpy as np

In [ ]:
#读取数据相关的内容
#读取coco数据集里的文件索引
import json
file_dic = {}
with open('datasets\instances_val2017.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    img_arr = json_data["images"]
    for i in range(len(img_arr)):
        img_name = img_arr[i]["file_name"]
        img_id = img_arr[i]["id"]
        file_dic[img_id] = img_name

coco_img_dir = "datasets/coco/images/val2017/"

In [5]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
# print(model)
model.eval()
clear_output()

In [6]:
# 定义 Pytorch 官方给的类别名称，有些是 'N/A' 是已经去掉的类别
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# coco_trans = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 13: 11, 
# 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 
# 25: 23, 27: 24, 28: 25, 31: 26, 32: 27, 33: 28, 34: 29, 35: 30, 36: 31, 37: 32, 38: 33, 
# 39: 34, 40: 35, 41: 36, 42: 37, 43: 38, 44: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 
# 51: 45, 52: 46, 53: 47, 54: 48, 55: 49, 56: 50, 57: 51, 58: 52, 59: 53, 60: 54, 61: 55, 
# 62: 56, 63: 57, 64: 58, 65: 59, 67: 60, 70: 61, 72: 62, 73: 63, 74: 64, 75: 65, 76: 66, 
# 77: 67, 78: 68, 79: 69, 80: 70, 81: 71, 82: 72, 84: 73, 85: 74, 86: 75, 87: 76, 88: 77, 
# 89: 78, 90: 79}

In [12]:
#  获取单张图片的预测结果
def get_prediction(img_path, file_id, threshold):
    img = Image.open(img_path)  # Load the image  加载图片
    transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
    img = transform(img)  # Apply the transform to the image  转换成 torch 形式
    pred = model([img.to(device)])  # Pass the image to the model  开始推理

    obj_list = []
    # for i in np.arange( len(pred[0]['labels']) ):
    for i in range(len(pred[0]['labels'])):
      img_obj = {}
      img_obj["image_id"] = file_id
      img_obj["category_id"] = int(pred[0]['labels'][i])
      img_obj["bbox"] = [float(i) for i in pred[0]['boxes'][i] ]
      img_obj["score"] = float(pred[0]['scores'][i])
      obj_list.append(img_obj)
    return obj_list

    #这些是为了画图做准备
    # pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())]  # Get the Prediction Score  获取预测的类别
    # pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().cpu().numpy())]  # Bounding boxes  获取各个类别的边框
    # pred_score = list(pred[0]['scores'].cpu().detach().numpy())  #  获取各个类别的分数

    # # Get list of index with score greater than threshold.
    # pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1]  #  判断分数大于阈值对于的分数的最大索引
    # #  因为预测后的分数是从大到小排序的，只要找到大于阈值最后一个的索引值即可
    # pred_boxes = pred_boxes[:pred_t+1]
    # pred_class = pred_class[:pred_t+1]
    # return pred_boxes, pred_class, obj_list

In [13]:
#  根据预测的结果绘制边框及类别
def object_detection_api(img_path, file_id, threshold=0.5, rect_th=1, text_size=0.5, text_th=1):
    boxes, pred_cls, imgboxs = get_prediction(img_path, file_id, threshold)  # Get predictions
    img = cv2.imread(img_path)  # Read image with cv2
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    result_dict = {}  #  用来保存每个类别的名称及数量
    for i in range(len(boxes)):
        color = tuple(random.randint(0, 255) for i in range(3))
        cv2.rectangle(img,
                      boxes[i][0],
                      boxes[i][1],
                      color=color,
                      thickness=rect_th)  # Draw Rectangle with the coordinates

        cv2.putText(img,
                    pred_cls[i],
                    boxes[i][0],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    text_size,
                    color,
                    thickness=text_th)  # Write the prediction class

        #  将各个预测的结果保存到一个字典里
        if pred_cls[i] not in result_dict:
            result_dict[pred_cls[i]] = 1
        else:
            result_dict[pred_cls[i]] += 1
        print(result_dict)
    plt.figure(figsize=(20, 30))  # display the output image
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    # plt.show()
    return imgboxs

In [ ]:
# list = object_detection_api("datasets\coco\images\val2017\000000000139.jpg" 000139)
coco_img_path = "/content/datasets/coco/images/val2017/000000000139.jpg"
file_id = 139
list = get_prediction(coco_img_path, file_id)
list

with open('val2017.json','a',encoding='utf8')as fp:
    json_data = json.dump(list, fp)

In [ ]:
result_list = []
for file_id in file_dic:
    coco_img_path = os.path.join(coco_img_dir,file_dic[file_id])
    list = object_detection_api(coco_img_path, file_id)
    result_list = np.append(result_list, list)
result_list = list(result_list)
